In [1]:
import os

import cvxpy as cp
import dill
import numpy as np
import pandas as pd

%matplotlib inline

os.chdir(os.path.expanduser('~/dev/vEcoli'))

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel
from ecoli.processes.registries import topology_registry
TOPOLOGY = topology_registry.access("ecoli-metabolism-redux")

%load_ext autoreload

In [2]:
# load checkpoint 2 model
time = '400'
date = '2025-05-15'
experiment = 'NEW_NewGenes_checkpoint2'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/metabolism-comparison/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

In [3]:
# get commonly stored variables
metabolism = agent['ecoli-metabolism-redux-classic']
stoichiometry = metabolism.stoichiometry.copy()
reaction_names = metabolism.reaction_names
fba_new_reaction_ids = metabolism.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism.parameters['fba_reaction_ids_to_base_reaction_ids']
metabolites = metabolism.metabolite_names.copy()
binary_kinetic_idx = metabolism.binary_kinetic_idx[0]
exchange_molecules = metabolism.exchange_molecules

S = stoichiometry.copy()
S = pd.DataFrame(S, index=metabolites , columns=reaction_names )
homeostatic_count = pd.DataFrame(fba["homeostatic_metabolite_counts"], columns=metabolism.homeostatic_metabolites).mean(axis=0)
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"], columns=metabolism.homeostatic_metabolites).mean(axis=0)
maintenance = pd.DataFrame(fba["maintenance_target"][1:], columns=['maintenance_reaction']).mean(axis=0)
kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).mean(axis=0).copy()

In [4]:
# parameters that are the same across the two simulation
kinetic_reaction_ids = metabolism.kinetic_constraint_reactions
allowed_exchange_uptake = metabolism.allowed_exchange_uptake
FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]
ADDED_RXNS = ['HS-TRANSPORT-RXN-CPD0-1202', 'HS-TRANSPORT-RXN-CPD0-1202 (reverse)',
                   'HS-TRANSPORT-RXN[CCO-OUTER-MEM]-OXAMATE', 'HS-TRANSPORT-RXN[CCO-OUTER-MEM]-OXAMATE (reverse)',
                   'HS-TRANSPORT-RXN[CCO-PM-BAC-NEG]-OXAMATE', 'HS-TRANSPORT-RXN[CCO-PM-BAC-NEG]-OXAMATE (reverse)',
                   'HS-BETA-GLUCURONID-RXN_CPD-3611//METOH', 'HS-SPONTANEOUS-TRANSPORT[CCO-OUTER-MEM]-HCN', 'HS-SPONTANEOUS-TRANSPORT[CCO-OUTER-MEM]-HCN (reverse)',
                   'HS-SPONTANEOUS-TRANSPORT[CCO-PM-BAC-NEG]-HCN','HS-SPONTANEOUS-TRANSPORT[CCO-PM-BAC-NEG]-HCN (reverse)']

# Define functions

In [5]:
def velocity_to_base_reaction(v):
    v = pd.DataFrame(v, index=reaction_names, columns=['flux'])
    v['base_reaction'] = v.index.map(fba_reaction_ids_to_base_reaction_ids)
    base_reaction_flux = v.groupby('base_reaction').sum()
    return base_reaction_flux

# Re-construct WC model's FBA problem

In [6]:
model = NetworkFlowModel(
            stoich_arr=S,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=metabolism.homeostatic_metabolites,
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
model.set_up_exchanges(exchanges=exchange_molecules, uptakes=metabolism.allowed_exchange_uptake)

In [7]:
objective_weights={'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.000001}

binary_kinetic_idx = binary_kinetic_idx
force_flow_idx = None

v = cp.Variable(model.n_orig_rxns)
e = cp.Variable(model.n_exch_rxns)
dm = model.S_orig @ v + model.S_exch @ e
exch = model.S_exch @ e

constr = []
constr.append(dm[model.intermediates_idx] == 0)

if model.maintenance_idx is not None:
    constr.append(v[model.maintenance_idx] == maintenance)
# If enzymes not present, constrain rxn flux to 0
if binary_kinetic_idx is not None:
    constr.append(v[binary_kinetic_idx] == 0)
# If want to force flow through reactions, constrain rxn flux to 1 by idx
if force_flow_idx is not None:
    constr.append(v[force_flow_idx] >= 100)

homeostatic_concs = homeostatic_count * metabolism.counts_to_molar.asNumber()
homeostatic_dm_targets=np.array(list(dict(homeostatic).values()))
kinetic_targets=np.array(list(dict(kinetic).values()))
upper_bound = 1000000000

constr.extend([v >= 0, v <= upper_bound, e >= 0, e <= upper_bound])

loss = 0
# Must normalize by metabolite concentrations to prevent negative counts
loss += cp.norm1(
    (dm[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs
)
# flux_sum_part_obj = objective_weights["secretion"] * (cp.sum(e[self.secretion_idx]))

loss += (
    objective_weights["secretion"] * (cp.sum(e[model.secretion_idx]))
    if "secretion" in objective_weights
    else loss
)
loss += (
    objective_weights["efficiency"] * (cp.sum(v))
    if "efficiency" in objective_weights
    else loss
)

loss = (
    loss
    + objective_weights["kinetics"]
    * cp.norm1(v[model.kinetic_rxn_idx] - kinetic_targets)
    if "kinetics" in objective_weights
    else loss
)

loss += (
    0.001 * cp.sum(cp.pos(100-v))
)

p = cp.Problem(cp.Minimize(loss), constr)
p.solve(solver=cp.GLOP, verbose=False)
print("status:", p.status)

status: optimal


objective: 108025.22625139347
number of nonzero velocities: 376
percentage of nonzero velocities: 4.011950490823731


In [65]:
print(objective_weights["efficiency"] * (cp.max(v)))

# result of changing efficiency term
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

1e-06 @ max(var118848, None, False)
objective: 108000.81988991922
number of nonzero fba reaction velocities: 511
number of nonzero base reaction velocities: 427
percentage of nonzero fba reaction velocities: 5.452411438326931
percentage of nonzero base reaction velocities: 15.260900643316655


In [66]:
np.max(velocities)

2257472.449663468

In [77]:
# result of changing efficiency and secretion term
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

objective: 102451.26223001747
number of nonzero fba reaction velocities: 563
number of nonzero base reaction velocities: 478
percentage of nonzero fba reaction velocities: 6.007255655142979
percentage of nonzero base reaction velocities: 17.08363116511794


In [87]:
# MOSEK MIP
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

objective: 102384.6717286675
number of nonzero fba reaction velocities: 6795
number of nonzero base reaction velocities: 1435
percentage of nonzero fba reaction velocities: 72.5032010243278
percentage of nonzero base reaction velocities: 51.28663330950679


In [10]:
# GLOP move away from sparse solution
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

objective: 108382.39703544292
number of nonzero fba reaction velocities: 6079
number of nonzero base reaction velocities: 1016
percentage of nonzero fba reaction velocities: 64.86342296201451
percentage of nonzero base reaction velocities: 36.311651179413865


In [16]:
np.max(velocities)

np.float64(3984773.436482362)

In [74]:
# Original problem
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

objective: 108025.22625097341
number of nonzero fba reaction velocities: 375
number of nonzero base reaction velocities: 366
percentage of nonzero fba reaction velocities: 4.001280409731113
percentage of nonzero base reaction velocities: 13.080771979985704


# Sweep through kinetic and non-sparse weights and see effect on number of reactions used

In [126]:
np.sum(np.abs(velocities[model.kinetic_rxn_idx] - kinetic_targets))

np.float64(53992398.08638623)

In [129]:
velocities

array([-0.0000000e+00,  1.0000000e+02,  1.0000000e+02, ...,
        9.0051200e+03,  1.0000000e+02,  1.9383935e+06], shape=(9372,))

In [133]:
np.sum(velocities)-np.sum(velocities[model.kinetic_rxn_idx])

np.float64(15916418.553050244)

In [26]:
# import kaleido
import plotly.graph_objects as go
# plot the weight of the term given a range of values and multiple alphas
n_alpha=8
alphas = np.logspace(2.5, 4, n_alpha)
weights = np.logspace(-5, -1, 1000)

kinetic_terms = weights * np.sum(np.abs(velocities[model.kinetic_rxn_idx] - kinetic_targets))
efficiency_terms = weights * np.sum(velocities)
efficiency_terms_max = weights * np.max(velocities)
secretion_terms = weights * np.sum(exchanges)
secretion_terms_max = weights * np.max(exchanges)

fig = go.Figure()

for i, alpha in enumerate(alphas):
    diversity_terms = weights * np.sum(np.maximum(alpha-velocities, 0))

    fig.add_trace(go.Scatter(x=weights, y=diversity_terms, mode='lines', name='Diversity term, α={alpha:.2g}'.format(alpha=alpha),
                             line=dict(dash='dash', color=f'rgba(44, 160, 44, {(i+1)/n_alpha})')))

fig.add_trace(go.Scatter(x=weights, y=kinetic_terms, mode='lines', name='Kinetic term', line=dict(color="#1f77b4")))
fig.add_trace(go.Scatter(x=weights, y=efficiency_terms, mode='lines', name='Efficiency term (cp.sum)', line=dict(color="brown")))
fig.add_trace(go.Scatter(x=weights, y=efficiency_terms_max, mode='lines', name='Efficiency term (cp.max)', line=dict(dash='dash', color="brown")))
fig.add_trace(go.Scatter(x=weights, y=secretion_terms, mode='lines', name='Secretion term (cp.sum)', line=dict(color="pink")))
fig.add_trace(go.Scatter(x=weights, y=secretion_terms_max, mode='lines', name='Secretion term (cp.max)', line=dict(dash='dash', color="pink")))


fig.update_xaxes(type='log', tickformat='~s', dtick=1)
fig.update_yaxes(type='log', tickformat='~s', dtick=1)
fig.update_layout(title=f'Kinetic and Diversity Terms vs Weights',
                  xaxis_title='Weight',
                  yaxis_title='Term Value',
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  height=600, width=1000)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/term_vs_weight.png", scale=3)

# Sweep through non-sparse weight and see effect on kinetic and diversity terms

In [150]:
from plotly.subplots import make_subplots
objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.0001}
# ---- parameters for the weights (so we can sweep without rebuilding) ----
w_secretion  = cp.Parameter(nonneg=True, value=objective_weights.get('secretion', 0.01))
w_efficiency = cp.Parameter(nonneg=True, value=objective_weights.get('efficiency', 0.000001))
w_kinetics   = cp.Parameter(nonneg=True, value=objective_weights.get('kinetics', 0.000001))
mu_diversity = cp.Parameter(nonneg=True, value=1e-3)      # we'll sweep this
alpha        = cp.Parameter(nonneg=True, value=3000.0)     # “activity floor”

# ---- objective terms kept separately so we can read their values after solve ----
homeo_term  = cp.norm1((dm[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs)
secretion_term  = cp.sum(e[model.secretion_idx])
efficiency_term = cp.sum(v)
kinetic_term    = cp.norm1(v[model.kinetic_rxn_idx] - kinetic_targets)
diversity_term  = cp.sum(cp.pos(alpha - v))               # non-sparse “hinge-away-from-zero”

loss = homeo_term + w_secretion*secretion_term + w_efficiency*efficiency_term \
       + w_kinetics*kinetic_term + mu_diversity*diversity_term

prob = cp.Problem(cp.Minimize(loss), constr)

# ---- sweep μ (non-sparse weight) and collect metric values ----
mu_grid = np.logspace(-6, -1, 15)   # adjust range as you like; includes your μ=1e-3
kin_vals  = []
diversity_vals = []
secretion_vals = []
efficiency_vals = []
statuses   = []

for mu in mu_grid:
    mu_diversity.value = float(mu)
    prob.solve(solver=cp.GLOP)
    statuses.append(prob.status)
    # If not optimal, append NaN to keep array lengths aligned
    kin_vals.append(float(kinetic_term.value) if prob.status in ("optimal", "optimal_inaccurate") else np.nan)
    diversity_vals.append(float(diversity_term.value)  if prob.status in ("optimal", "optimal_inaccurate") else np.nan)
    secretion_vals.append(float(secretion_term.value)  if prob.status in ("optimal", "optimal_inaccurate") else np.nan)
    efficiency_vals.append(float(efficiency_term.value)  if prob.status in ("optimal", "optimal_inaccurate") else np.nan)

/Users/HeenaSaqib/dev/vEcoli/.venv/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:241: UserWarning:

You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html



In [151]:
# ---- one figure, two subplots, each has two y-axes, log-scaled x ----
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=mu_grid, y=kin_vals,  mode="lines+markers",
                         name="Kinetic L1 (‖v_kin−target‖₁)", line=dict(color="#1f77b4")),
              secondary_y=False)
fig.add_trace(go.Scatter(x=mu_grid, y=diversity_vals, mode="lines+markers",
                         name="Diversity term (∑max(α−v,0))", line=dict(dash="dash", color="rgba(44, 160, 44, 1)")),
              secondary_y=True)

fig.update_xaxes(title_text="weight on the diversity term", type="log")
fig.update_yaxes(title_text="Kinetic L1", secondary_y=False)
fig.update_yaxes(title_text="Diversity term", secondary_y=True)
fig.update_layout(title="Tradeoff vs weight on diversity term",
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  width=800, height=450)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_sweep_diversity_a.png", scale=3)

In [29]:
# ---- one figure, two subplots, each has two y-axes, log-scaled x ----
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=mu_grid, y=secretion_vals,  mode="lines+markers",
                         name="Secretion term(∑exchanges)", line=dict(dash="dot", color='#20735d')),
              secondary_y=False)
fig.add_trace(go.Scatter(x=mu_grid, y=efficiency_vals, mode="lines+markers",
                         name="Efficiency term (∑v)", line=dict(dash="dash", color='brown')),
              secondary_y=True)

fig.update_xaxes(title_text="weight on the diversity term", type="log")
fig.update_yaxes(title_text="Secretion term", secondary_y=False)
fig.update_yaxes(title_text="Efficiency term", secondary_y=True)
fig.update_layout(title="Tradeoff vs weight on diversity term",
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  width=800, height=450)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_sweep_diversity_b.png", scale=3)

In [31]:
# ---- one figure, two subplots, each has two y-axes, log-scaled x ----
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=mu_grid, y=secretion_vals,  mode="lines+markers",
                         name="Secretion term(max(exchanges))", line=dict(dash="dot", color='#20735d')),
              secondary_y=False)
fig.add_trace(go.Scatter(x=mu_grid, y=efficiency_vals, mode="lines+markers",
                         name="Efficiency term (max(v))", line=dict(dash="dash", color='brown')),
              secondary_y=True)

fig.update_xaxes(title_text="weight on the diversity term", type="log")
fig.update_yaxes(title_text="Secretion term", secondary_y=False)
fig.update_yaxes(title_text="Efficiency term", secondary_y=True)
fig.update_layout(title="Tradeoff vs weight on diversity term",
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  width=800, height=450)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_sweep_diversity_c.png", scale=3)

# Sweep through alpha and see effect on kinetic and diversity terms

In [36]:
from plotly.subplots import make_subplots

# ---- parameters for the weights (so we can sweep without rebuilding) ----
w_secretion  = cp.Parameter(nonneg=True, value=objective_weights.get('secretion', 0.01))
w_efficiency = cp.Parameter(nonneg=True, value=objective_weights.get('efficiency', 0.000001))
w_kinetics   = cp.Parameter(nonneg=True, value=0.00001)
mu_diversity = cp.Parameter(nonneg=True, value=1e-3)      # we'll sweep this
alpha_        = cp.Parameter(nonneg=True, value=3000.0)     # “activity floor”

# ---- objective terms kept separately so we can read their values after solve ----
homeo_term  = cp.norm1((dm[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs)
secretion_term  = cp.max(e[model.secretion_idx])  #TODO: change these to lines to max or sum depending on what we want
efficiency_term = cp.max(v) #TODO: change these to lines to max or sum depending on what we want
kinetic_term    = cp.norm1(v[model.kinetic_rxn_idx] - kinetic_targets)
diversity_term      = cp.sum(cp.pos(alpha_ - v))               # non-sparse “hinge-away-from-zero”

loss = homeo_term + w_secretion*secretion_term + w_efficiency*efficiency_term \
       + w_kinetics*kinetic_term + mu_diversity*diversity_term

prob = cp.Problem(cp.Minimize(loss), constr)

# ---- sweep alpha and collect metric values ----
alpha_grid = np.logspace(0, 5, 15)
kin_vals  = []
diversity_vals = []
secretion_vals = []
efficiency_vals = []
statuses   = []

for alpha in alpha_grid:
    alpha_.value = float(alpha)
    prob.solve(solver=cp.GLOP)
    statuses.append(prob.status)
    # If not optimal, append NaN to keep array lengths aligned
    kin_vals.append(float(kinetic_term.value) if prob.status in ("optimal", "optimal_inaccurate") else np.nan)
    diversity_vals.append(float(diversity_term.value)  if prob.status in ("optimal", "optimal_inaccurate") else np.nan)
    secretion_vals.append(float(secretion_term.value)  if prob.status in ("optimal", "optimal_inaccurate") else np.nan)
    efficiency_vals.append(float(efficiency_term.value)  if prob.status in ("optimal", "optimal_inaccurate") else np.nan)

/Users/HeenaSaqib/dev/vEcoli/.venv/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:241: UserWarning:

You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html



In [34]:
# ---- one figure, two subplots, each has two y-axes, log-scaled x ----
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=alpha_grid, y=kin_vals,  mode="lines+markers",
                         name="Kinetic L1 (‖v_kin−target‖₁)", line=dict(color="#1f77b4")),
              secondary_y=False)
fig.add_trace(go.Scatter(x=alpha_grid, y=diversity_vals, mode="lines+markers",
                         name="Diversity term (∑max(α−v,0))", line=dict(dash="dash", color="rgba(44, 160, 44, 1)")),
              secondary_y=True)

fig.update_xaxes(title_text="alpha in the diversity term", type="log")
fig.update_yaxes(title_text="Kinetic L1", secondary_y=False)
fig.update_yaxes(title_text="Diversity term", secondary_y=True)
fig.update_layout(title="Tradeoff vs alpha in the diversity term",
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  width=800, height=450)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_sweep_alpha_a.png", scale=3)

In [35]:
# ---- one figure, two subplots, each has two y-axes, log-scaled x ----
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=alpha_grid, y=secretion_vals,  mode="lines+markers",
                         name="Secretion term(∑exchanges)", line=dict(dash="dot", color='#20735d')),
              secondary_y=False)
fig.add_trace(go.Scatter(x=alpha_grid, y=efficiency_vals, mode="lines+markers",
                         name="Efficiency term (∑v)", line=dict(dash="dash", color='brown')),
              secondary_y=True)

fig.update_xaxes(title_text="alpha in the diversity term", type="log")
fig.update_yaxes(title_text="Secretion term", secondary_y=False)
fig.update_yaxes(title_text="Efficiency term", secondary_y=True)
fig.update_layout(title="Tradeoff vs alpha in the diversity term",
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  width=800, height=450)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_sweep_alpha_b.png", scale=3)

In [37]:
# ---- one figure, two subplots, each has two y-axes, log-scaled x ----
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=mu_grid, y=secretion_vals,  mode="lines+markers",
                         name="Secretion term(max(exchanges))", line=dict(dash="dot", color='#20735d')),
              secondary_y=False)
fig.add_trace(go.Scatter(x=mu_grid, y=efficiency_vals, mode="lines+markers",
                         name="Efficiency term (max(v))", line=dict(dash="dash", color='brown')),
              secondary_y=True)

fig.update_xaxes(title_text="alpha in the diversity term", type="log")
fig.update_yaxes(title_text="Secretion term", secondary_y=False)
fig.update_yaxes(title_text="Efficiency term", secondary_y=True)
fig.update_layout(title="Tradeoff vs alpha in the diversity term",
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  width=800, height=450)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_sweep_alpha_c.png", scale=3)

# Sweep through kinetic weight and see effect on kinetic and diversity terms

In [41]:
from plotly.subplots import make_subplots

# ---- parameters for the weights (so we can sweep without rebuilding) ----
w_secretion  = cp.Parameter(nonneg=True, value=objective_weights.get('secretion', 0.01))
w_efficiency = cp.Parameter(nonneg=True, value=objective_weights.get('efficiency', 0.000001))
w_kinetics   = cp.Parameter(nonneg=True, value=0.00001)
mu_diversity = cp.Parameter(nonneg=True, value=1e-3)      # we'll sweep this
alpha_        = cp.Parameter(nonneg=True, value=2000.0)     # “activity floor”

# ---- objective terms kept separately so we can read their values after solve ----
homeo_term  = cp.norm1((dm[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs)
secretion_term  = cp.max(e[model.secretion_idx])
efficiency_term = cp.max(v)
kinetic_term    = cp.norm1(v[model.kinetic_rxn_idx] - kinetic_targets)
diversity_term      = cp.sum(cp.pos(alpha_ - v))               # non-sparse “hinge-away-from-zero”

loss = homeo_term + w_secretion*secretion_term + w_efficiency*efficiency_term \
       + w_kinetics*kinetic_term + mu_diversity*diversity_term

prob = cp.Problem(cp.Minimize(loss), constr)

# ---- sweep alpha and collect metric values ----
w_kinetics_grid = np.logspace(-6, 0, 15)
kin_vals  = []
diversity_vals = []
secretion_vals = []
efficiency_vals = []
statuses   = []

for w_kinetic in w_kinetics_grid:
    w_kinetics.value = float(w_kinetic)
    prob.solve(solver=cp.GLOP)
    statuses.append(prob.status)
    # If not optimal, append NaN to keep array lengths aligned
    kin_vals.append(float(kinetic_term.value) if prob.status in ("optimal", "optimal_inaccurate") else np.nan)
    diversity_vals.append(float(diversity_term.value)  if prob.status in ("optimal", "optimal_inaccurate") else np.nan)
    secretion_vals.append(float(secretion_term.value)  if prob.status in ("optimal", "optimal_inaccurate") else np.nan)
    efficiency_vals.append(float(efficiency_term.value)  if prob.status in ("optimal", "optimal_inaccurate") else np.nan)

/Users/HeenaSaqib/dev/vEcoli/.venv/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:241: UserWarning:

You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html



In [39]:
# ---- one figure, two subplots, each has two y-axes, log-scaled x ----
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=w_kinetics_grid, y=kin_vals,  mode="lines+markers",
                         name="Kinetic L1 (‖v_kin−target‖₁)", line=dict(color="#1f77b4")),
              secondary_y=False)
fig.add_trace(go.Scatter(x=w_kinetics_grid, y=diversity_vals, mode="lines+markers",
                         name="Diversity term (∑max(α−v,0))", line=dict(dash="dash", color="rgba(44, 160, 44, 1)")),
              secondary_y=True)

fig.update_xaxes(title_text="Weight on the kinetic term", type="log")
fig.update_yaxes(title_text="Kinetic L1", secondary_y=False)
fig.update_yaxes(title_text="Diversity term", secondary_y=True)
fig.update_layout(title="Tradeoff vs Weight on the kinetic term",
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  width=800, height=450)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_sweep_kinetic_a.png", scale=3)

In [40]:
# ---- one figure, two subplots, each has two y-axes, log-scaled x ----
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=w_kinetics_grid, y=secretion_vals,  mode="lines+markers",
                         name="Secretion term(∑exchanges)", line=dict(dash="dot", color='#20735d')),
              secondary_y=False)
fig.add_trace(go.Scatter(x=w_kinetics_grid, y=efficiency_vals, mode="lines+markers",
                         name="Efficiency term (∑v)", line=dict(dash="dash", color='brown')),
              secondary_y=True)

fig.update_xaxes(title_text="Weight on the kinetic term", type="log")
fig.update_yaxes(title_text="Secretion term", secondary_y=False)
fig.update_yaxes(title_text="Efficiency term", secondary_y=True)
fig.update_layout(title="Tradeoff vs Weight on the kinetic term",
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  width=800, height=450)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_sweep_kinetic_b.png", scale=3)

In [42]:
# ---- one figure, two subplots, each has two y-axes, log-scaled x ----
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=w_kinetics_grid, y=secretion_vals,  mode="lines+markers",
                         name="Secretion term(max(exchanges))", line=dict(dash="dot", color='#20735d')),
              secondary_y=False)
fig.add_trace(go.Scatter(x=w_kinetics_grid, y=efficiency_vals, mode="lines+markers",
                         name="Efficiency term (max(v))", line=dict(dash="dash", color='brown')),
              secondary_y=True)

fig.update_xaxes(title_text="weight on the kinetic term", type="log")
fig.update_yaxes(title_text="Secretion term", secondary_y=False)
fig.update_yaxes(title_text="Efficiency term", secondary_y=True)
fig.update_layout(title="Tradeoff vs weight on kinetic term",
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  width=800, height=450)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_sweep_kinetic_c.png", scale=3)

# Plot 3D surface of kinetic vs diversity vs alpha terms

In [48]:
import numpy as np
import cvxpy as cp
import plotly.graph_objects as go

objective_weights = {'secretion': 0.01, 'efficiency': 0.000001}
# ---------- Build the problem ONCE (use Parameters for weights) ----------
n_v, n_e = model.n_orig_rxns, model.n_exch_rxns
v = cp.Variable(n_v, nonneg=True)
e = cp.Variable(model.n_exch_rxns, nonneg=True)

dm = model.S_orig @ v + model.S_exch @ e

constr = [dm[model.intermediates_idx] == 0]
if model.maintenance_idx is not None:
    constr += [v[model.maintenance_idx] == maintenance]
if binary_kinetic_idx is not None:
    constr += [v[binary_kinetic_idx] == 0]

upper_bound = 1_000_000_000.0
constr += [v <= upper_bound, e <= upper_bound]

# Fixed weights (from your dict); these can be Parameters too if you want to sweep them.
w_sec  = objective_weights.get('secretion', 0.0)
w_eff  = objective_weights.get('efficiency', 0.0)

# Parameters to sweep
lam_kin = cp.Parameter(nonneg=True, value=1e-6)   # kinetic weight
mu      = cp.Parameter(nonneg=True, value=1e-3)   # diversity/dense weight
alpha   = cp.Parameter(nonneg=True, value=100.0)  # activity floor for dense term

# Objective pieces
homeo_term   = cp.norm1((dm[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs)
secretion    = cp.sum(e[model.secretion_idx])
efficiency   = cp.sum(v)
kin_term     = cp.norm1(v[model.kinetic_rxn_idx] - kinetic_targets)
dense_term   = cp.sum(cp.pos(alpha - v))          # encourages many v_i >= alpha

loss = homeo_term + w_sec*secretion + w_eff*efficiency + lam_kin*kin_term + mu*dense_term
prob = cp.Problem(cp.Minimize(loss), constr)

# ---------- Define the sweep grids ----------
KIN_GRID   = np.logspace(-6, 0, 10)   # λ_kin
MU_GRID    = np.logspace(-6, 0, 10)   # μ
ALPHA_GRID = np.logspace(2, 4, 10)     # α  (e.g., 1e2..1e4)
COUNT_THRESH = 1e-9                    # what counts as "non-zero" reaction

# success
X, Y, Z = [], [], []
support_counts, obj_values = [], []

# non-optimal (infeasible/unbounded)
X_bad, Y_bad, Z_bad = [], [], []

# exceptions (solver error)
X_err, Y_err, Z_err = [], [], []

for L in KIN_GRID:
    lam_kin.value = float(L)
    for M in MU_GRID:
        mu.value = float(M)
        for A in ALPHA_GRID:
            alpha.value = float(A)
            try:
                prob.solve(solver=cp.GLOP, warm_start=True, verbose=False)
            except cp.SolverError:
                # record solver crash as a black point
                X_err.append(np.log10(L)); Y_err.append(np.log10(M)); Z_err.append(np.log10(A))
                continue

            # solver returned a status without crashing
            if prob.status not in ("optimal", "optimal_inaccurate"):
                # plot as gray
                X_bad.append(np.log10(L)); Y_bad.append(np.log10(M)); Z_bad.append(np.log10(A))
                continue

            # success: collect metrics
            vv = np.asarray(v.value).ravel()
            X.append(np.log10(L)); Y.append(np.log10(M)); Z.append(np.log10(A))
            support_counts.append(int((vv >= COUNT_THRESH).sum()))
            obj_values.append(float(prob.value))


/Users/HeenaSaqib/dev/vEcoli/.venv/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:241: UserWarning:

You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html



In [55]:
X = np.array(X); Y = np.array(Y); Z = np.array(Z)
support_counts = np.array(support_counts, dtype=float)
obj_values = np.array(obj_values, dtype=float)
obj_log = np.log10(np.clip(obj_values, 1e-30, None))

# --- Plotly traces ---
trace_support = go.Scatter3d(
    x=X, y=Y, z=Z, mode='markers',
    marker=dict(size=4, color=support_counts, colorscale='Viridis',
                colorbar=dict(title='# active rxns')),
    name='# Active', visible=True
)
trace_obj = go.Scatter3d(
    x=X, y=Y, z=Z, mode='markers',
    marker=dict(size=4, color=obj_log, colorscale='Viridis',
                colorbar=dict(title='log₁₀(Objective)')),
    name='Objective', visible=False
)
trace_bad = go.Scatter3d(
    x=X_bad, y=Y_bad, z=Z_bad, mode='markers',
    marker=dict(size=4, color='gray', opacity=0.8),
    name='Not optimal', showlegend=True, visible=True
)
trace_err = go.Scatter3d(
    x=X_err, y=Y_err, z=Z_err, mode='markers',
    marker=dict(size=5, color='black', symbol='x'),
    name='Solver error', showlegend=True, visible=True
)

# mark minimum objective among successes
if len(obj_values) > 0:
    i_min = int(np.nanargmin(obj_values))
    trace_best = go.Scatter3d(
        x=[-5], y=[-4], z=[3], mode='markers+text',
        marker=dict(size=8, color='red', symbol='diamond'),
        text=["Optimum"], textposition='top center',
        name='Optimum', visible=True
    )
else:
    trace_best = go.Scatter3d(x=[], y=[], z=[], mode='markers', visible=False)

fig = go.Figure(data=[trace_support, trace_bad, trace_err, trace_best])
fig.update_layout(
    title='3D sweep: log₁₀(λ_kin) vs log₁₀(λ_div) vs log₁₀(α)',
    scene=dict(
        xaxis_title='log₁₀(kinetic weight λ_kin)',
        yaxis_title='log₁₀(diversity weight λ_div)',
        zaxis_title='log₁₀(targeted minimal flow α)'
    ),
    width=950, height=700,
    updatemenus=[dict(
        type="buttons", showactive=True, x=1.05, y=1,
        buttons=[
            dict(label="Color: # active reactions", method="update",
                 args=[{"visible": [True, False, True, True, True]},
                       {"title": "Color = # active reactions"}]),
            dict(label="Color: log₁₀(objective)", method="update",
                 args=[{"visible": [False, True, True, True, True]},
                       {"title": "Color = log₁₀(objective)"}]),
        ]
    )],
    template='plotly_white',
    paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
    xaxis=dict(
        gridcolor='lightgray'  # Set x-axis grid color
    ),
    yaxis=dict(
        gridcolor='lightgray'  # Set y-axis grid color
    ),)
fig.show()


# Plot flux distribution before and after adding diversity term

In [43]:
objective_weights={'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.000001, 'diversity': 0.00001}
alpha = 1000

binary_kinetic_idx = binary_kinetic_idx
force_flow_idx = None

v = cp.Variable(model.n_orig_rxns)
e = cp.Variable(model.n_exch_rxns)
dm = model.S_orig @ v + model.S_exch @ e
exch = model.S_exch @ e

constr = []
constr.append(dm[model.intermediates_idx] == 0)

if model.maintenance_idx is not None:
    constr.append(v[model.maintenance_idx] == maintenance)
# If enzymes not present, constrain rxn flux to 0
if binary_kinetic_idx is not None:
    constr.append(v[binary_kinetic_idx] == 0)
# If want to force flow through reactions, constrain rxn flux to 1 by idx
if force_flow_idx is not None:
    constr.append(v[force_flow_idx] >= 100)

homeostatic_concs = homeostatic_count * metabolism.counts_to_molar.asNumber()
homeostatic_dm_targets=np.array(list(dict(homeostatic).values()))
kinetic_targets=np.array(list(dict(kinetic).values()))
upper_bound = 1000000000

constr.extend([v >= 0, v <= upper_bound, e >= 0, e <= upper_bound])

loss = 0
# Must normalize by metabolite concentrations to prevent negative counts
loss += cp.norm1(
    (dm[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs
)
# flux_sum_part_obj = objective_weights["secretion"] * (cp.sum(e[self.secretion_idx]))

loss += (
    objective_weights["secretion"] * (cp.sum(e[model.secretion_idx]))
    if "secretion" in objective_weights
    else loss
)
loss += (
    objective_weights["efficiency"] * (cp.sum(v))
    if "efficiency" in objective_weights
    else loss
)

loss = (
    loss
    + objective_weights["kinetics"]
    * cp.norm1(v[model.kinetic_rxn_idx] - kinetic_targets)
    if "kinetics" in objective_weights
    else loss
)

# loss += (
#     objective_weights["diversity"] * cp.sum(cp.pos(alpha-v))
# )

p = cp.Problem(cp.Minimize(loss), constr)
p.solve(solver=cp.GLOP, verbose=False)
print("status:", p.status)

status: optimal


In [42]:
# GLOP move away from sparse solution
velocities_after = np.array(v.value)
dm_dt_after = np.array(dm.value)
exchanges_after = np.array(exch.value)
objective_after = p.value

print("objective:", objective_after)
print("number of nonzero fba reaction velocities:", np.sum(velocities_after > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities_after)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities_after > 1e-6) / len(velocities_after)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities_after)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities_after))*100)
print(max(velocities_after))

objective: 108068.61644864122
number of nonzero fba reaction velocities: 5741
number of nonzero base reaction velocities: 821
percentage of nonzero fba reaction velocities: 61.2569355527102
percentage of nonzero base reaction velocities: 29.34238741958542
3988462.6782451603


In [40]:
# GLOP move away from sparse solution with max terms
velocities_max = np.array(v.value)
dm_dt_max = np.array(dm.value)
exchanges_max = np.array(exch.value)
objective_max = p.value

print("objective:", objective_max)
print("number of nonzero fba reaction velocities:", np.sum(velocities_max > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities_max)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities_max > 1e-6) / len(velocities_max)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities_max)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities_max))*100)
print(max(velocities_max))

objective: 123500.9837539767
number of nonzero fba reaction velocities: 5736
number of nonzero base reaction velocities: 820
percentage of nonzero fba reaction velocities: 61.20358514724712
percentage of nonzero base reaction velocities: 29.306647605432453
1938393.5025


In [44]:
# Before adding diversity term
velocities = np.array(v.value)
dm_dt = np.array(dm.value)
exchanges = np.array(exch.value)
objective = p.value

print("objective:", objective)
print("number of nonzero fba reaction velocities:", np.sum(velocities > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities > 1e-6) / len(velocities)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))*100)

objective: 108025.22625104697
number of nonzero fba reaction velocities: 372
number of nonzero base reaction velocities: 365
percentage of nonzero fba reaction velocities: 3.969270166453265
percentage of nonzero base reaction velocities: 13.045032165832737


np.float64(57285.762828990104)

In [45]:
# plot distribution of reaction flux
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- inputs ---
v_before = np.asarray(velocities, dtype=float).ravel()
v_after  = np.asarray(velocities_after, dtype=float).ravel()
v_max    = np.asarray(velocities_max, dtype=float).ravel()
NONZERO_THRESH = 1e-6  # your threshold for "active"

# --- split zeros vs positives (for hist we log-transform positives) ---

# Common binning in log10 space
xb = np.log10(v_before + NONZERO_THRESH)
xa = np.log10(v_after + NONZERO_THRESH)
xc = np.log10(v_max + NONZERO_THRESH)
xmin = float(np.floor(min(xb.min(), xa.min(), xc.min())))
xmax = float(np.ceil (max(xb.max(), xa.max(), xc.max())))
nbins = 50

# --- figure with two panels ---
fig = go.Figure()

# Left: overlaid histograms (log10(flux))
fig.add_trace(
    go.Histogram(x=xb, nbinsx=nbins, name="Original FBA", opacity=0.55)
)
fig.add_trace(
    go.Histogram(x=xa, nbinsx=nbins, name="Adding Diversity", opacity=0.55)
)
fig.add_trace(
    go.Histogram(x=xc, nbinsx=nbins, name="Adding Diversity + max()", opacity=0.55)
)

fig.update_xaxes(title_text="log₁₀(flux + 1e-6)", range=[xmin, xmax])
fig.update_yaxes(title_text="Count")
fig.update_layout(barmode="overlay",
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(
                      gridcolor='lightgray'  # Set x-axis grid color
                  ),
                  yaxis=dict(
                      gridcolor='lightgray'  # Set y-axis grid color
                  ),
                  height=600, width=1000)  # overlap the two hists


fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_flux_distribution.png", scale=3)

In [16]:
print(np.max(velocities_max), np.max(velocities_after))
print(np.max(exchanges), np.max(exchanges_after))
print(np.sum(velocities), np.sum(velocities_after))
print(objective_max, objective_after)

homeos = cp.norm(
    (dm_dt_after[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs
)
homeos.value

5234463.474577908 3988462.6782451603
3987468.7532451986 3987468.7532451604
20646991.093737952 26402649.703738138
102488.4522594671 108068.61644864122


np.float64(57285.762828990104)

In [46]:
# plot distribution of reaction flux
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("number of nonzero fba reaction velocities:", np.sum(velocities_after > 1e-6))
print("number of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities_after)['flux'] > 1e-6))
print("percentage of nonzero fba reaction velocities:", np.sum(velocities_after > 1e-6) / len(velocities_after)*100)
print("percentage of nonzero base reaction velocities:", np.sum(velocity_to_base_reaction(velocities_after)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities_after))*100)

# --- inputs ---
max_velocity = np.max(velocities)
max_exchanges = np.max(exchanges)
sum_fluxes   = np.sum(velocities)
homeostatic_flux = cp.norm((dm_dt[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs).value
num_nonzero_fba_reactions = np.sum(velocities > 1e-6)
num_nonzero_base_reactions = np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6)
perc_nonzero_fba_reactions = np.sum(velocities > 1e-6) / len(velocities)
perc_nonzero_base_reactions = np.sum(velocity_to_base_reaction(velocities)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities))

max_velocity_after = np.max(velocities_after)
max_exchanges_after = np.max(exchanges_after)
sum_fluxes_after   = np.sum(velocities_after)
homeostatic_flux_after = cp.norm((dm_dt_after[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs).value
num_nonzero_fba_reactions_after = np.sum(velocities_after > 1e-6)
num_nonzero_base_reactions_after = np.sum(velocity_to_base_reaction(velocities_after)['flux'] > 1e-6)
perc_nonzero_fba_reactions_after = np.sum(velocities_after > 1e-6) / len(velocities_after)
perc_nonzero_base_reactions_after = np.sum(velocity_to_base_reaction(velocities_after)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities_after))

max_velocity_max = np.max(velocities_max)
max_exchanges_max = np.max(exchanges_max)
sum_fluxes_max   = np.sum(velocities_max)
homeostatic_flux_max = cp.norm((dm_dt_max[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs).value
num_nonzero_fba_reactions_max = np.sum(velocities_max > 1e-6)
num_nonzero_base_reactions_max = np.sum(velocity_to_base_reaction(velocities_max)['flux'] > 1e-6)
perc_nonzero_fba_reactions_max = np.sum(velocities_max > 1e-6) / len(velocities_max)
perc_nonzero_base_reactions_max = np.sum(velocity_to_base_reaction(velocities_max)['flux'] > 1e-6) / len(velocity_to_base_reaction(velocities_max))


metrics = ["Max reaction flux", "Max exchange flux", "Efficiency term (Σ(v))", "Homeostatic term", "Objective Function Value", "# FBA reactions used", "# Base reactions used", "% FBA reactions used", "% Base reactions used"]
before  = [max_velocity, max_exchanges, sum_fluxes, homeostatic_flux, objective, num_nonzero_fba_reactions, num_nonzero_base_reactions, perc_nonzero_fba_reactions, perc_nonzero_base_reactions]
after   = [max_velocity_after, max_exchanges_after, sum_fluxes_after, homeostatic_flux_after, objective_after, num_nonzero_fba_reactions_after,
           num_nonzero_base_reactions_after, perc_nonzero_fba_reactions_after, perc_nonzero_base_reactions_after]
after_max     = [max_velocity_max, max_exchanges_max, sum_fluxes_max, homeostatic_flux_max, objective_max, num_nonzero_fba_reactions_max,
                 num_nonzero_base_reactions_max, perc_nonzero_fba_reactions_max, perc_nonzero_base_reactions_max]

fig = go.Figure()

# Before (no diversity term)
fig.add_trace(go.Bar(
    x=metrics[:5], y=before[:5], name="Original FBA",
    text=[f"{v:.3g}" for v in before[:5]], textposition="outside", marker_color='salmon'
))

# After (with diversity term)
fig.add_trace(go.Bar(
    x=metrics[:5], y=after[:5], name="Adding Diversity",
    text=[f"{v:.3g}" for v in after[:5]], textposition="outside", marker_color='cadetblue'
))

# After_max (with diversity term and max() instead of sum() secretion and efficiency)
fig.add_trace(go.Bar(
    x=metrics[:5], y=after_max[:5], name="Adding Diversity + max()",
    text=[f"{v:.3g}" for v in after_max[:5]], textposition="outside", marker_color='burlywood'
))

fig.update_layout(
    barmode="group",
    template='plotly_white',
    paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
    xaxis=dict(gridcolor='lightgray'),
    yaxis=dict(gridcolor='lightgray', title="Value", type='log'), #plot on log scale
    height=600, width=1000
)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_flux_metrics_a.png", scale=3)

number of nonzero fba reaction velocities: 5741
number of nonzero base reaction velocities: 821
percentage of nonzero fba reaction velocities: 61.2569355527102
percentage of nonzero base reaction velocities: 29.34238741958542


In [47]:
# ---- one figure, two subplots, each has two y-axes, log-scaled x ----
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Before (no diversity term)
fig.add_trace(go.Bar(
    x=metrics[5:7], y=before[5:7], name="Original FBA",
    text=[f"{v:d}" for v in before[5:7]], textposition="outside", marker_color='salmon'
), secondary_y=False)

# After (with diversity term)
fig.add_trace(go.Bar(
    x=metrics[5:7], y=after[5:7], name="Adding Diversity",
    text=[f"{v:d}" for v in after[5:7]], textposition="outside", marker_color='cadetblue'
), secondary_y=False)

# After_max (with diversity term and max() instead of sum() secretion and efficiency)
fig.add_trace(go.Bar(
    x=metrics[5:7], y=after_max[5:7], name="Adding Diversity + max()",
    text=[f"{v:d}" for v in after_max[5:7]], textposition="outside", marker_color='burlywood'
), secondary_y=False)

# Before (no diversity term)
fig.add_trace(go.Bar(
    x=metrics[7:], y=before[7:],
    text=[f"{v:.2%}" for v in before[7:]], textposition="outside", marker_color='salmon',
), secondary_y=True)

# After (with diversity term)
fig.add_trace(go.Bar(
    x=metrics[7:], y=after[7:],
    text=[f"{v:.2%}" for v in after[7:]], textposition="outside", marker_color='cadetblue'
), secondary_y=True)

# After_max (with diversity term and max() instead of sum() secretion and efficiency)
fig.add_trace(go.Bar(
    x=metrics[7:], y=after_max[7:],
    text=[f"{v:.2%}" for v in after_max[7:]], textposition="outside", marker_color='burlywood'
), secondary_y=True)

#
# fig.add_trace(go.Scatter(x=mu_grid, y=kin_vals,  mode="lines+markers",
#                          name="Kinetic L1 (‖v_kin−target‖₁)", line=dict(color="#1f77b4")),
#               secondary_y=False)
# fig.add_trace(go.Scatter(x=mu_grid, y=diversity_vals, mode="lines+markers",
#                          name="Diversity term (∑max(α−v,0))", line=dict(dash="dash", color="rgba(44, 160, 44, 1)")),
#               secondary_y=True)

fig.update_layout(
    barmode="group",
    template='plotly_white',
    paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
    xaxis=dict(gridcolor='lightgray'),
    yaxis=dict(gridcolor='lightgray', title="Value"),
    height=600, width=1200,
    legend_title_text=""
)
fig.show()
fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/FBA_flux_metrics_b.png", scale=3)

In [102]:
homeostatic_concs

2-3-DIHYDROXYBENZOATE[c]    0.132206
2-KETOGLUTARATE[c]          0.338178
2-PG[c]                     0.087945
2K-4CH3-PENTANOATE[c]       0.132206
4-AMINO-BUTYRATE[c]         0.291236
                              ...   
NA+[p]                      0.095801
OXYGEN-MOLECULE[p]          0.095801
FE+3[p]                     0.095801
CA+2[p]                     0.095801
Pi[p]                       0.095801
Length: 172, dtype: float64